In [1]:
import pandas as pd
import numpy as np
from numpy import linalg as LA
import copy
import sys

In this part we Load the Data points

In [2]:
Test_data = pd.read_csv('/content/drive/MyDrive/DCI Project/DataSet/testData.csv', header = None)
Test_label = pd.read_csv('/content/drive/MyDrive/DCI Project/DataSet/testLabels.csv', header = None)
Train_data = pd.read_csv('/content/drive/MyDrive/DCI Project/DataSet/trainData.csv', header = None)
Train_label = pd.read_csv('/content/drive/MyDrive/DCI Project/DataSet/trainLabels.csv', header = None)

Converting the Data frames to numpy matrix

In [3]:
Test_data_np = Test_data.to_numpy()
Test_label_np = Test_label.to_numpy()
Train_data_np = Train_data.to_numpy()
Train_label_np = Train_label.to_numpy()

The function that calculates the covariance Matrix (Needed to find the Eigen vectors)

In [4]:
def calc_cov(data_set, mean):
    res = np.zeros([data_set.shape[1],data_set.shape[1]])
    for i in range(data_set.shape[0]):
        vect = np.subtract(data_set[i], mean)
        vect = np.array([vect])
        res += np.matmul(vect.T, vect)

    return res/(data_set.shape[0]-1)

The Node class which represents every node on the Lines

In [5]:
class Node:
  def __init__(self, dataval, idx):
    self.dataval = dataval
    self.nextval = None
    self.preval = None
    self.idx = idx

We have Linked list class which we are going to use as the line

In [6]:
class SLinkedList:
  def __init__(self, headnode, condition="Node", the_list=None):
    self.headnode = headnode
    self.lastnode = headnode
    self.star = None
    if condition == "list":
      temp = the_list.headnode
      temp = temp.nextval
      while True:
        self.add_node(Node(temp.dataval, temp.idx))
        if temp.nextval == None:
          break
        temp = temp.nextval
  
  def add_node(self, node):
    self.lastnode.nextval = node
    node.preval = self.lastnode
    self.lastnode = node

  def print_node(self):
    current = self.headnode
    while True:
      current = current.nextval
      if current == None:
        break

  def add_star(self, value):
    temp = self.headnode
    star_node = Node(value, -1)
    self.star = star_node
    if temp.dataval > value:
      self.star.nextval = self.headnode
      self.headnode.preval = self.star
      self.headnode = self.star
      return 
    while True:
      if temp.nextval == None:
        temp.nextval  = self.star
        self.star.preval = temp
        return
      if temp.dataval <= value and temp.nextval.dataval > value:
        self.star.nextval = temp.nextval
        temp.nextval.preval = self.star
        temp.nextval = self.star
        self.star.preval = temp
        return
      temp = temp.nextval

  def close_val(self):
    if self.star.nextval == None:
      return (self.star.dataval - self.star.preval.dataval), "left"
    if self.star.preval == None:
      return (self.star.nextval.dataval - self.star.dataval), "right"
    temp1 = (self.star.nextval.dataval - self.star.dataval)
    temp2 = (self.star.dataval - self.star.preval.dataval)
    if temp1 >= temp2:
      return temp2, "left"
    else:
      return temp1, "right"

  def get_rid_close(self, direction):
    if direction == "right":
      key = self.star.nextval.idx
      if self.star.nextval.nextval == None:
        self.star.nextval = None
        return key
      self.star.nextval.nextval.preval = self.star
      self.star.nextval = self.star.nextval.nextval
      return key
    if direction == "left":
      key = self.star.preval.idx
      if self.star.preval.preval == None:
        self.star.preval = None
        return key
      self.star.preval.preval.nextval = self.star
      self.star.preval = self.star.preval.preval
      return key
  
  def add_new_data(self, line, label):
    temp = self.headnode
    flag = 0 
    if temp.dataval > line[0]:
      new = Node(line[0], label[0])
      new.nextval = temp
      temp.preval = new
      self.headnode = new
      flag = 1
      temp = self.headnode
    for i in range(flag, len(label)):
      while True:
        if temp.nextval == None:
          new = Node(line[i], label[i])
          new.preval = temp
          temp.nextval = new
          break
        if temp.dataval < line[i] and temp.nextval.dataval > line[i]:
          new = Node(line[i], label[i])
          new.nextval = temp.nextval
          temp.nextval.preval = new
          temp.nextval = new
          new.preval = temp
          temp = temp.nextval
          break
        else:
          temp = temp.nextval
          continue
    print("done this")
    
  def delete_node(self, line):
    temp = self.headnode
    flag = 0
    if temp.dataval == line[0]:
      temp.nextval.preval = None
      self.headnode = temp.nextval
      flag = 1
      temp = temp.nextval
    for i in range(flag, len(line)):
      while True:
        if temp.dataval == line[i]:
          if temp.preval != None:
            temp.preval.nextval = temp.nextval
          if temp.nextval != None:
            temp.nextval.preval = temp.preval
          break
        if temp.dataval > line[i]:
          break
        else:
          temp = temp.nextval
    print("Delete Done")




Make Like function, by giving a sorted vector to it, we will get a line which every element is on it

In [7]:
def make_line(sorted_data):
  first = Node(sorted_data[0][0], sorted_data[0][1])
  result = SLinkedList(first)
  for i in range(1, sorted_data.shape[0]):
    current = Node(sorted_data[i][0], sorted_data[i][1])
    result.add_node(current)
  return result

The DCI model Class

In [8]:
class New_DCI:
  def __init__(self, number_line, Train_data_np, Train_label):
    self.Ultimate_DataBase = Train_data_np
    self.Ultimate_LabelBase = Train_label
    self.number_line = number_line
    self.The_Lines = []
    self.The_Lines_Direction = []
    self.num_data = Train_data_np.shape[0]
    Data_avg = np.mean(Train_data_np, axis = 0)
    Data_cov = calc_cov(Train_data_np, Data_avg)
    EigenValues, EigenVectors = LA.eigh(Data_cov) 
    self.New_EigenVectors = EigenVectors[:,number_line:]
    print("Done for Eigen vectors")
    for i in range(number_line):
      print(i)
      current_line = (np.dot(Train_data_np, self.New_EigenVectors[:, i]))
      self.The_Lines_Direction.append(self.New_EigenVectors[:, i])
      p = current_line.argsort()
      line = current_line[p]
      label = (np.arange(len(Train_label)).reshape(-1, 1))[p]
      res = np.concatenate((line.reshape(-1, 1), label), axis=1)
      self.The_Lines.append(make_line(res))
    self.The_Lines_Direction = np.array(self.The_Lines_Direction)

  def insert_new_data(self, Data, Data_label):
    for i in range(self.number_line):
      current_line = (np.dot(Data, self.New_EigenVectors[:, i]))
      p = current_line.argsort()
      line = current_line[p]
      label = (np.arange(self.num_data, self.num_data + len(Data_label)).reshape(-1, 1))[p]
      self.The_Lines[i].add_new_data(line, label)
    self.Ultimate_DataBase = np.concatenate((self.Ultimate_DataBase, Data), axis=0)
    self.Ultimate_LabelBase = np.concatenate((self.Ultimate_LabelBase, Data_label), axis=0)
    self.num_data += len(Data_label)
    print("Added complete")
    

  def delete_data(self, Data):
    for i in range(self.number_line):
      current_line = (np.dot(Data, self.New_EigenVectors[:, i]))
      p = current_line.argsort()
      line = current_line[p]
      self.The_Lines[i].delete_node(line)


The functions for testing the Model

In [9]:
def Test_DCI(model, Test_data_np, number_points):
  The_Keys = []
  for i in range(5): # This part is just for test we are going to test only 5 data points
    current = np.sum(Test_data_np[i].reshape(1, -1) * model.The_Lines_Direction, axis=1)
    The_Keys.append(Find_best_point(model.The_Lines, number_points, current, model.number_line))
  return The_Keys

In [10]:
def Find_best_point(Lines, number_point, current, number_line):
  current_point = 0
  The_Lines_copy = []
  for j in range(number_line):
    One_Line = SLinkedList(Node(Lines[j].headnode.dataval, Lines[j].headnode.idx), "list", Lines[j])
    The_Lines_copy.append(One_Line)
  finish_key = []
  my_dict = {}
  for i in range(len(The_Lines_copy)):
    The_Lines_copy[i].add_star(current[i])

  while True:
    direction = []
    dist = []
    for i in range(len(The_Lines_copy)):
      temp_dist, temp_dir = The_Lines_copy[i].close_val()
      dist.append(temp_dist)
      direction.append(temp_dir)

    min_arg = np.argmin(np.array(dist))
    key = The_Lines_copy[min_arg].get_rid_close(direction[min_arg])
    key = int(key)
    if key in my_dict.keys():
      my_dict[key] += 1
      if my_dict[key] == len(The_Lines_copy):
        current_point += 1
        finish_key.append(key)
    else:
      my_dict[key] = 1

    if current_point == number_point:
      print("done")
      return finish_key

    
  



Now we will test it

In [11]:
MyDCI = New_DCI(10, Train_data_np, Train_label)

Done for Eigen vectors
0
1
2
3
4
5
6
7
8
9


In [12]:
Final_result = Test_DCI(MyDCI, Test_data_np, 100)

done
done
done
done
done
done
done
done
done
done


For 5 data points

In [21]:
from sklearn.neighbors import KNeighborsClassifier
for k in range(5):
  neigh = KNeighborsClassifier(n_neighbors=3)
  temp = [int(i) for i in Final_result[k]]
  neigh.fit(Train_data_np[temp], Train_label_np[temp])
  print("The new DCI method for ", k, "data example is:", neigh.predict(Test_data_np[k].reshape(1, -1)))

The new DCI method for  0 data example is: [9]
The new DCI method for  1 data example is: [2]
The new DCI method for  2 data example is: [1]
The new DCI method for  3 data example is: [1]
The new DCI method for  4 data example is: [0]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-

In [14]:
from sklearn.neighbors import KNeighborsClassifier

neigh2 = KNeighborsClassifier(n_neighbors=3)
neigh2.fit(Train_data_np, Train_label_np)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  after removing the cwd from sys.path.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [20]:
for k in range(5):
  print("KNN for all data prediction: ", neigh2.predict(Test_data_np[k].reshape(1, -1)), " Real label: ", Test_label_np[k])

KNN for all data prediction:  [9]  Real label:  [9]
KNN for all data prediction:  [2]  Real label:  [2]
KNN for all data prediction:  [1]  Real label:  [1]
KNN for all data prediction:  [1]  Real label:  [1]
KNN for all data prediction:  [6]  Real label:  [6]


In [22]:
MyDCI.insert_new_data(Test_data_np, Test_label_np)

done this
done this
done this
done this
done this
done this
done this
done this
done this
done this
Added complete


In [23]:
MyDCI.delete_data(Test_data_np)

Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
Delete Done
